# Análise de demonstrativos financeiros
## Indicadores de Rentabilidade

## 1. Carrega os pacotes

In [24]:
# Carrega as bibliotecas
!pip install wget
!pip install plotnine
import wget # biblioteca para realizar o download de sites
import pandas as pd # biblioteca para manipulação de dados e data frames
from zipfile import ZipFile # biblioteca para extrair arquivo zipados
from plotnine import * # biblioteca para construir o gráfico

## 2. Cria função para coletar demonstrativos da CVM

In [25]:
def download_extract_concatenate_dfp_files(start_year, end_year, financial_statements):
    """
    Função para realizar o download, extração e concatenação de arquivos zipados contendo os dados da Demonstração Financeira
    Padronizada (DFP) de companhias abertas disponibilizados pela Comissão de Valores Mobiliários (CVM).

    Parâmetros:
    start_year (int): Ano inicial da coleta dos dados.
    end_year (int): Ano final da coleta dos dados.
    financial_statements (list): Lista de nomes dos arquivos de demonstrativos financeiros que deseja-se concatenar. Consolidado: BPA_con (balanço patrimonial ativo);
     BPP_con (Balanço patrimonial passivo) DRE_con; DFC_MI_con; DFC_MD_CON; DMPL_con.


    Retorna:
    Arquivo .csv no diretório principal
    """
    url = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/"
    # Cria uma lista vazia para inserir os nomes dos arquivos zipados
    arquivo_zipado = []

    # Define os nomes dos arquivos zipados de acordo com o range de datas
    for ano in range(start_year, end_year + 1):
        arquivo_zipado.append(f'dfp_cia_aberta_{ano}.zip')

    # Realiza o download dos arquivos zipados de acordo com a url base
    for arquivos in arquivo_zipado:
        wget.download(url + arquivos)

    # Extrai os arquivos zipados
    for arquivos in arquivo_zipado:
        ZipFile(arquivos, 'r').extractall('DFP')

    # Concatena os dados dos demonstrativos financeiros em um único DataFrame
    for demons in financial_statements:
        arquivo_demonstrativo = pd.DataFrame()
        for ano in range(start_year, end_year + 1):
            arquivo_demonstrativo = pd.concat([arquivo_demonstrativo, pd.read_csv(f'DFP/dfp_cia_aberta_{demons}_{ano}.csv', sep = ';', decimal = ',', encoding = 'ISO-8859-1')])
        arquivo_demonstrativo.to_csv(f'dfp_cia_aberta_{demons}_{start_year}-{end_year}.csv', index = False)

## 3. Coleta os dados

In [26]:
# Coleta os dados dos demonstrativos
download_extract_concatenate_dfp_files(2010, 2023, ['BPA_con', 'BPP_con', 'DRE_con'])

In [27]:
# Realiza a leitura do arquivo
dre = pd.read_csv('/content/dfp_cia_aberta_DRE_con_2010-2023.csv')

In [28]:
# Realiza a leitura do arquivo
bpa = pd.read_csv('/content/dfp_cia_aberta_BPA_con_2010-2023.csv')

In [29]:
# Realiza a leitura do arquivo
bpp = pd.read_csv('/content/dfp_cia_aberta_BPP_con_2010-2023.csv')

## 4. Tratamento de dados

In [30]:
# Junta os dados
dados = pd.concat([bpa, bpp, dre], axis = 0)

In [31]:
dados

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,DT_INI_EXERC
0,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,PENÚLTIMO,2009-12-31,1,Ativo Total,7.025720e+08,S,NaN
1,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2010-12-31,1,Ativo Total,8.028198e+08,S,NaN
2,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,PENÚLTIMO,2009-12-31,1.01,Caixa e Equivalentes de Caixa,3.103788e+07,S,NaN
3,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2010-12-31,1.01,Caixa e Equivalentes de Caixa,2.521970e+07,S,NaN
4,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,PENÚLTIMO,2009-12-31,1.01.01,Caixa e Depósitos Bancários,8.098625e+06,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384831,97.837.181/0001-47,2023-12-31,1,DEXCO S.A.,21091,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2023-12-31,3.99.01.01,ON,9.772000e-01,N,2023-01-01
384832,97.837.181/0001-47,2023-12-31,1,DEXCO S.A.,21091,DF Consolidado - Demonstração do Resultado,REAL,MIL,PENÚLTIMO,2022-12-31,3.99.02,Lucro Diluído por Ação,0.000000e+00,N,2022-01-01
384833,97.837.181/0001-47,2023-12-31,1,DEXCO S.A.,21091,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2023-12-31,3.99.02,Lucro Diluído por Ação,0.000000e+00,N,2023-01-01
384834,97.837.181/0001-47,2023-12-31,1,DEXCO S.A.,21091,DF Consolidado - Demonstração do Resultado,REAL,MIL,PENÚLTIMO,2022-12-31,3.99.02.01,ON,1.014400e+00,N,2022-01-01


In [32]:
casan= dados.loc[dados['DENOM_CIA']=='CASAN S.A']

In [33]:
casan

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,DT_INI_EXERC


In [34]:
# Realiza o filtro das contas contábeis, do código CVM e da ordem do exercício
dados = dados[dados.CD_CONTA.isin(["1", # Ativo Total
                                "2", # Passivo total
                                "2.03", # Patrimônio Líquido
                                "3.05", # EBIT
                                "3.08", # Imposto de Renda e Contribuição Social sobre o Lucro
                                "3.11"  # Lucro/Prejuízo Consolidado do Período
                                 ]) & dados.CD_CVM.isin([1023, 26700, 21431, 19550]) & (dados.ORDEM_EXERC == 'ÚLTIMO')]


In [35]:
# Seleciona as colunas
dados = dados[["DT_REFER", "DENOM_CIA", "CD_CONTA", "DS_CONTA", "VL_CONTA"]]

In [36]:
dados

,DT_REFER,DENOM_CIA,CD_CONTA,DS_CONTA,VL_CONTA
1,2010-12-31,BCO BRASIL S.A.,1,Ativo Total,802819794.0
8998,2010-12-31,HYPERA S.A.,1,Ativo Total,9987182.0
38805,2010-12-31,NATURA COSMETICOS S.A.,1,Ativo Total,3221871.0
46019,2011-12-31,BCO BRASIL S.A.,1,Ativo Total,966823068.0
54209,2011-12-31,HYPERA S.A.,1,Ativo Total,13325286.0
...,...,...,...,...,...
379251,2023-12-31,EUROFARMA LABORATÓRIOS S.A.,3.08,Imposto de Renda e Contribuição Social sobre o...,-88731.0
379265,2023-12-31,EUROFARMA LABORATÓRIOS S.A.,3.11,Lucro/Prejuízo Consolidado do Período,649698.0
380553,2023-12-31,NATURA COSMETICOS S.A.,3.05,Resultado Antes do Resultado Financeiro e dos ...,1980386.0
380565,2023-12-31,NATURA COSMETICOS S.A.,3.08,Imposto de Renda e Contribuição Social sobre o...,588234.0


In [37]:
# Converte a coluna de data para datetime
dados['DT_REFER'] = pd.to_datetime(dados['DT_REFER'])

# Cria a coluna Ano
dados['Ano'] = dados['DT_REFER'].dt.year

dados_pivot = dados.pivot(index = ['Ano', 'DENOM_CIA'], columns = 'DS_CONTA', values = 'VL_CONTA')

In [38]:
dados_pivot.head()

DS_CONTA                     Ativo Total  \
Ano  DENOM_CIA                             
2010 BCO BRASIL S.A.         802819794.0   
     HYPERA S.A.               9987182.0   
     NATURA COSMETICOS S.A.    3221871.0   
2011 BCO BRASIL S.A.         966823068.0   
     HYPERA S.A.              13325286.0   

DS_CONTA                     Imposto de Renda e Contribuição Social sobre o Lucro  \
Ano  DENOM_CIA                                                                      
2010 BCO BRASIL S.A.                                                       NaN      
     HYPERA S.A.                                                     -166825.0      
     NATURA COSMETICOS S.A.                                          -374120.0      
2011 BCO BRASIL S.A.                                                       NaN      
     HYPERA S.A.                                                       14595.0      

DS_CONTA                     Lucro ou Prejuízo Líquido Consolidado do Período  \
Ano  DENOM_CIA                                                                  
2010 BCO BRASIL S.A.                                                      NaN   
     HYPERA S.A.                                                          NaN   
     NATURA COSMETICOS S.A.                                               NaN   
2011 BCO BRASIL S.A.                                                      NaN   
     HYPERA S.A.                                                          NaN   

DS_CONTA                     Lucro/Prejuízo Consolidado do Período  \
Ano  DENOM_CIA                                                       
2010 BCO BRASIL S.A.                                           NaN   
     HYPERA S.A.                                          261901.0   
     NATURA COSMETICOS S.A.                               744050.0   
2011 BCO BRASIL S.A.                                           NaN   
     HYPERA S.A.                                          -54651.0   

DS_CONTA                     Passivo Total  \
Ano  DENOM_CIA                               
2010 BCO BRASIL S.A.           802819794.0   
     HYPERA S.A.                 9987182.0   
     NATURA COSMETICOS S.A.      3221871.0   
2011 BCO BRASIL S.A.           966823068.0   
     HYPERA S.A.                13325286.0   

DS_CONTA                     Passivos Financeiros ao Custo Amortizado  \
Ano  DENOM_CIA                                                          
2010 BCO BRASIL S.A.                                      626821814.0   
     HYPERA S.A.                                                  NaN   
     NATURA COSMETICOS S.A.                                       NaN   
2011 BCO BRASIL S.A.                                      774043490.0   
     HYPERA S.A.                                                  NaN   

DS_CONTA                     Patrimônio Líquido Consolidado  Provisões  \
Ano  DENOM_CIA                                                           
2010 BCO BRASIL S.A.                                    NaN        NaN   
     HYPERA S.A.                                  5059091.0        NaN   
     NATURA COSMETICOS S.A.                       1257502.0        NaN   
2011 BCO BRASIL S.A.                                    NaN        NaN   
     HYPERA S.A.                                  6645934.0        NaN   

DS_CONTA                     Resultado Antes do Resultado Financeiro e dos Tributos  \
Ano  DENOM_CIA                                                                        
2010 BCO BRASIL S.A.                                                       NaN        
     HYPERA S.A.                                                      611956.0        
     NATURA COSMETICOS S.A.                                          1167906.0        
2011 BCO BRASIL S.A.                                                       NaN        
     HYPERA S.A.                                                      421009.0        

DS_CONTA                     Resultado Antes dos Tributos sobre o Lucro 

## 5. Cria os indicadores

In [39]:
# Cria os indicadores
indicadores = (
  dados_pivot
 .assign(roic = (dados_pivot["Resultado Antes do Resultado Financeiro e dos Tributos"] - dados_pivot["Imposto de Renda e Contribuição Social sobre o Lucro"]) / dados_pivot["Passivo Total"] * 100,
         roe = (dados_pivot["Lucro/Prejuízo Consolidado do Período"]) / dados_pivot["Patrimônio Líquido Consolidado"] * 100,
         roa = (dados_pivot["Lucro/Prejuízo Consolidado do Período"]) / dados_pivot["Ativo Total"] * 100)
  )

# Retira o índice
indicadores.reset_index(inplace = True)

In [40]:
# Arrendonda os valores
indicadores = indicadores.round(decimals = 2)
indicadores.head()

DS_CONTA,Ano,DENOM_CIA,Ativo Total,Imposto de Renda e Contribuição Social sobre o Lucro,Lucro ou Prejuízo Líquido Consolidado do Período,Lucro/Prejuízo Consolidado do Período,Passivo Total,Passivos Financeiros ao Custo Amortizado,Patrimônio Líquido Consolidado,Provisões,Resultado Antes do Resultado Financeiro e dos Tributos,Resultado Antes dos Tributos sobre o Lucro,Resultado Líquido das Operações Descontinuadas,Resultado antes dos Tributos sobre o Lucro,roic,roe,roa
0,2010,BCO BRASIL S.A.,802819794.0,NaN,NaN,NaN,802819794.0,626821814.0,NaN,NaN,NaN,16550866.0,0.0,NaN,NaN,NaN,NaN
1,2010,HYPERA S.A.,9987182.0,-166825.0,NaN,261901.0,9987182.0,NaN,5059091.0,NaN,611956.0,NaN,NaN,NaN,7.80,5.18,2.62
2,2010,NATURA COSMETICOS S.A.,3221871.0,-374120.0,NaN,744050.0,3221871.0,NaN,1257502.0,NaN,1167906.0,NaN,NaN,NaN,47.86,59.17,23.09
3,2011,BCO BRASIL S.A.,966823068.0,NaN,NaN,NaN,966823068.0,774043490.0,NaN,NaN,NaN,17795255.0,0.0,NaN,NaN,NaN,NaN
4,2011,HYPERA S.A.,13325286.0,14595.0,NaN,-54651.0,13325286.0,NaN,6645934.0,NaN,421009.0,NaN,NaN,NaN,3.05,-0.82,-0.41


In [41]:
# Seleciona colunas
indicadores = indicadores[['Ano', 'DENOM_CIA', 'roic', 'roe', 'roa']]

In [42]:
indicadores.head()

DS_CONTA,Ano,DENOM_CIA,roic,roe,roa
0,2010,BCO BRASIL S.A.,NaN,NaN,NaN
1,2010,HYPERA S.A.,7.80,5.18,2.62
2,2010,NATURA COSMETICOS S.A.,47.86,59.17,23.09
3,2011,BCO BRASIL S.A.,NaN,NaN,NaN
4,2011,HYPERA S.A.,3.05,-0.82,-0.41


In [43]:
# Definindo cores
colors = {
    "blue": "#282f6b",
    "red": "#b22200",
    "yellow": "#eace3f",
    "green": "#224f20",
    "purple": "#5f487c",
    "orange": "#b35c1e",
    "turquoise": "#419391",
    "green_two": "#839c56",
    "light_blue": "#3b89bc",
    "gray": "#666666"
}

In [44]:
import plotly.express as px

In [45]:
fig = px.line(indicadores, x="Ano", y="roic", color="DENOM_CIA",
              facet_col="DENOM_CIA", facet_col_wrap=2, color_discrete_sequence=list(colors.values()),
              labels={"Ano":"", "roic":""}, title="ROIC de empresas do setor de Farmacêutico e Higiene")

fig.update_layout(height=500, width=900, showlegend=False)
fig.show()